# Week 2 Homework: Project Dataset Preprocessing and Feature Engineering (Draft)

## Business Context
FinMark Corporation currently offers standard financial products such as savings accounts, credit cards, and loans. However, these offerings lack personalization and often fail to align with the unique needs and preferences of individual customers.

## Objective
This homework prepares FinMark’s datasets for customer segmentation by performing data preprocessing (cleaning, standardisation, merging) and feature engineering (aggregations, ratios, and time-based features). The output will serve as the foundation for next week’s validated dataset and for Milestone 1 (EDA Report).

## Datasets
- Transaction_Data.csv  
- Product_Offering_Data.csv  
- Customer_Feedback_Data.csv


In [1]:
import pandas as pd
import numpy as np

from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 200)
pd.set_option("display.width", 120)

In [2]:
TRANSACTION_PATH = "Transaction_Data.csv"
PRODUCT_PATH = "Product_Offering_Data.csv"
FEEDBACK_PATH = "Customer_Feedback_Data.csv"

df_txn = pd.read_csv(TRANSACTION_PATH)
df_prod = pd.read_csv(PRODUCT_PATH)
df_fb = pd.read_csv(FEEDBACK_PATH)

print("Loaded:")
print("Transactions:", df_txn.shape)
print("Products:", df_prod.shape)
print("Feedback:", df_fb.shape)

Loaded:
Transactions: (5050, 5)
Products: (15, 6)
Feedback: (5050, 4)


In [3]:
def standardise_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = (
        df.columns
          .str.strip()
          .str.lower()
          .str.replace(" ", "_")
          .str.replace("-", "_")
          .str.replace("__", "_")
    )
    return df

df_txn = standardise_columns(df_txn)
df_prod = standardise_columns(df_prod)
df_fb = standardise_columns(df_fb)

df_txn.columns.tolist(), df_prod.columns.tolist(), df_fb.columns.tolist()

(['transaction_id',
  'customer_id',
  'transaction_date',
  'transaction_amount',
  'transaction_type'],
 ['product_id',
  'product_name',
  'product_type',
  'risk_level',
  'target_age_group',
  'target_income_group'],
 ['customer_id',
  'satisfaction_score',
  'feedback_comments',
  'likelihood_to_recommend'])

## Step 1: Initial Inspection (Structure, Types, Basic Stats)

In this section, I inspect each dataset to understand:
- shape, column types, and basic summary statistics
- missing values and duplicates
- likely key identifiers (e.g., customer_id, product_id)
- likely date fields (e.g., transaction_date, feedback_date)
- numeric anomalies (e.g., negative amounts, extreme values)

No rows are removed yet in this step—this is only for diagnosis.

In [4]:
def missing_summary(df: pd.DataFrame) -> pd.DataFrame:
    ms = df.isna().mean().sort_values(ascending=False)
    out = pd.DataFrame({
        "missing_pct": (ms * 100).round(2),
        "missing_count": df.isna().sum()
    })
    return out[out["missing_count"] > 0]

def duplicate_count(df: pd.DataFrame) -> int:
    return df.duplicated().sum()

def quick_audit(df: pd.DataFrame, name: str):
    print("="*90)
    print(f"{name} | shape={df.shape}")
    print("="*90)
    display(df.head(3))
    print("\nINFO:")
    display(df.info())
    print("\nDESCRIBE (include all):")
    display(df.describe(include="all").transpose().head(30))
    print("\nMISSING SUMMARY:")
    ms = missing_summary(df)
    display(ms if not ms.empty else pd.DataFrame({"missing_pct": [], "missing_count": []}))
    print("\nDUPLICATE ROW COUNT:", duplicate_count(df))
    print("\nCANDIDATE ID COLUMNS (contains 'id'):")
    id_cols = [c for c in df.columns if "id" in c]
    print(id_cols)
    if id_cols:
        display(df[id_cols].nunique().to_frame("nunique"))
    print()

quick_audit(df_txn, "Transaction Data")
quick_audit(df_prod, "Product Offering Data")
quick_audit(df_fb, "Customer Feedback Data")


Transaction Data | shape=(5050, 5)


,transaction_id,customer_id,transaction_date,transaction_amount,transaction_type
0,1,393,2023-01-01 00:00:00,3472.0,Purchase
1,2,826,2023-01-01 01:00:00,NaN,Bill Payment
2,3,916,2023-01-01 02:00:00,10.0,Purchase



INFO:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5050 entries, 0 to 5049
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   transaction_id      5050 non-null   int64  
 1   customer_id         5050 non-null   int64  
 2   transaction_date    5050 non-null   object 
 3   transaction_amount  4950 non-null   float64
 4   transaction_type    5050 non-null   object 
dtypes: float64(1), int64(2), object(2)
memory usage: 197.4+ KB


None


DESCRIBE (include all):


...

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
customer_id,5050.0,NaN,NaN,NaN,501.949703,288.806331,1.0,253.0,502.0,751.75,1000.0
satisfaction_score,4949.0,NaN,NaN,NaN,5.684785,3.606511,1.0,3.0,6.0,8.0,60.0
feedback_comments,5050,5,Good service,1519,NaN,NaN,NaN,NaN,NaN,NaN,NaN
likelihood_to_recommend,5050.0,NaN,NaN,NaN,5.571287,2.872577,1.0,3.0,6.0,8.0,10.0



MISSING SUMMARY:


,missing_pct,missing_count
satisfaction_score,2.0,101



DUPLICATE ROW COUNT: 81

CANDIDATE ID COLUMNS (contains 'id'):
['customer_id']


,nunique
customer_id,1000


In [5]:
def find_first_existing(df: pd.DataFrame, candidates: list[str]) -> str | None:
    for c in candidates:
        if c in df.columns:
            return c
    return None

# candidate keys
customer_id_col_txn = find_first_existing(df_txn, ["customer_id", "cust_id", "client_id", "customerid"])
product_id_col_txn  = find_first_existing(df_txn, ["product_id", "prod_id", "productid"])
txn_id_col          = find_first_existing(df_txn, ["transaction_id", "txn_id", "trans_id", "transactionid"])

customer_id_col_fb  = find_first_existing(df_fb, ["customer_id", "cust_id", "client_id", "customerid"])
product_id_col_prod = find_first_existing(df_prod, ["product_id", "prod_id", "productid"])

print("Detected columns:")
print("Txn customer id:", customer_id_col_txn)
print("Txn product id:", product_id_col_txn)
print("Txn transaction id:", txn_id_col)
print("Feedback customer id:", customer_id_col_fb)
print("Product product id:", product_id_col_prod)

Detected columns:
Txn customer id: customer_id
Txn product id: None
Txn transaction id: transaction_id
Feedback customer id: customer_id
Product product id: product_id


In [6]:
def parse_dates_inplace(df: pd.DataFrame, candidate_date_cols: list[str]) -> pd.DataFrame:
    df = df.copy()
    for c in candidate_date_cols:
        if c in df.columns:
            df[c] = pd.to_datetime(df[c], errors="coerce")
    return df

# Add/adjust candidates if your dataset uses different naming
date_candidates_txn = ["transaction_date", "date", "txn_date", "timestamp", "created_at"]
date_candidates_fb  = ["feedback_date", "date", "created_at", "submitted_at"]
date_candidates_prod = ["launch_date", "created_at", "date"]

df_txn = parse_dates_inplace(df_txn, date_candidates_txn)
df_fb  = parse_dates_inplace(df_fb, date_candidates_fb)
df_prod= parse_dates_inplace(df_prod, date_candidates_prod)

# Print min/max for any parsed date columns
def date_minmax(df, name):
    date_cols = [c for c in df.columns if np.issubdtype(df[c].dtype, np.datetime64)]
    print(f"{name} date columns:", date_cols)
    for c in date_cols:
        print(f" - {c}: min={df[c].min()} | max={df[c].max()}")

date_minmax(df_txn, "Transaction")
date_minmax(df_fb, "Feedback")
date_minmax(df_prod, "Product")

Transaction date columns: ['transaction_date']
 - transaction_date: min=2023-01-01 00:00:00 | max=2023-07-28 07:00:00
Feedback date columns: []
Product date columns: []


## Step 2: Data Preprocessing (Cleaning and Standardisation)

Cleaning is performed with business logic and modelling readiness in mind:
- Remove fully duplicated rows
- Ensure numeric fields are numeric and free of formatting symbols
- Standardise categorical text fields (lowercase, trim spaces)
- Handle missing values with careful rules:
  - transaction amount or transaction date missing → remove (cannot infer behaviour reliably)
  - missing product category/type → fill as "unknown"
  - missing feedback rating → keep record but exclude from rating-derived features

In [7]:
df_txn = df_txn.drop_duplicates()
df_prod = df_prod.drop_duplicates()
df_fb = df_fb.drop_duplicates()

print("After drop_duplicates:")
print("Transactions:", df_txn.shape)
print("Products:", df_prod.shape)
print("Feedback:", df_fb.shape)

After drop_duplicates:
Transactions: (5000, 5)
Products: (10, 6)
Feedback: (4969, 4)


In [8]:
def to_numeric_clean(series: pd.Series) -> pd.Series:
    # remove commas, currency symbols, and whitespace
    return pd.to_numeric(
        series.astype(str)
              .str.replace(",", "", regex=False)
              .str.replace("₱", "", regex=False)
              .str.replace("$", "", regex=False)
              .str.strip(),
        errors="coerce"
    )

def clean_numeric_columns(df: pd.DataFrame, numeric_candidates: list[str]) -> pd.DataFrame:
    df = df.copy()
    for c in numeric_candidates:
        if c in df.columns:
            df[c] = to_numeric_clean(df[c])
    return df

# Adjust candidate lists after you inspect your columns
txn_numeric_candidates = ["amount", "transaction_amount", "txn_amount", "balance", "account_balance"]
fb_numeric_candidates = ["rating", "score", "feedback_rating"]
prod_numeric_candidates = ["fee", "interest_rate", "apr", "min_balance"]

df_txn = clean_numeric_columns(df_txn, txn_numeric_candidates)
df_fb  = clean_numeric_columns(df_fb, fb_numeric_candidates)
df_prod= clean_numeric_columns(df_prod, prod_numeric_candidates)

df_txn.head(3)

,transaction_id,customer_id,transaction_date,transaction_amount,transaction_type
0,1,393,2023-01-01 00:00:00,3472.0,Purchase
1,2,826,2023-01-01 01:00:00,NaN,Bill Payment
2,3,916,2023-01-01 02:00:00,10.0,Purchase


In [9]:
def clean_text(series: pd.Series) -> pd.Series:
    return (series.astype(str)
            .str.strip()
            .str.lower()
            .replace("nan", np.nan))

def clean_categorical_columns(df: pd.DataFrame, max_unique_ratio: float = 0.5) -> pd.DataFrame:
    """
    Clean object columns that are likely categorical (not free-text).
    We apply cleaning if unique values are relatively small vs row count.
    """
    df = df.copy()
    obj_cols = df.select_dtypes(include=["object"]).columns.tolist()
    for c in obj_cols:
        nunique = df[c].nunique(dropna=True)
        if len(df) == 0:
            continue
        if nunique / len(df) <= max_unique_ratio:
            df[c] = clean_text(df[c])
    return df

df_txn = clean_categorical_columns(df_txn)
df_prod = clean_categorical_columns(df_prod)
df_fb = clean_categorical_columns(df_fb)

In [10]:
# Identify likely amount and date cols in transactions
amount_col = find_first_existing(df_txn, ["transaction_amount", "amount", "txn_amount"])
txn_date_col = find_first_existing(df_txn, ["transaction_date", "date", "txn_date", "timestamp", "created_at"])

print("Using amount_col =", amount_col)
print("Using txn_date_col =", txn_date_col)

# Rule: transaction amount or date missing => drop
if amount_col is not None:
    df_txn = df_txn[df_txn[amount_col].notna()]
if txn_date_col is not None:
    df_txn = df_txn[df_txn[txn_date_col].notna()]

# Product missing category/type => fill "unknown"
prod_type_col = find_first_existing(df_prod, ["product_type", "type", "category", "product_category"])
print("Using prod_type_col =", prod_type_col)
if prod_type_col is not None:
    df_prod[prod_type_col] = df_prod[prod_type_col].fillna("unknown")

# Feedback missing rating: keep but rating-based features will ignore NaN
rating_col = find_first_existing(df_fb, ["feedback_rating", "rating", "score"])
print("Using rating_col =", rating_col)

print("After missing handling:")
print("Transactions:", df_txn.shape)
print("Products:", df_prod.shape)
print("Feedback:", df_fb.shape)

Using amount_col = transaction_amount
Using txn_date_col = transaction_date
Using prod_type_col = product_type
Using rating_col = None
After missing handling:
Transactions: (4900, 5)
Products: (10, 6)
Feedback: (4969, 4)


## Step 3: Dataset Integration Decision (Analytical Justification)

After inspecting the datasets, no valid product-level foreign key (e.g., product_id)
exists in the transaction data. Transactions represent customer-level financial
behaviour rather than purchases of specific products.

Therefore:
- Transaction data will NOT be merged with product offering data
- Product offering data will be used later for recommendation mapping
- Customer segmentation will be based on behavioural and feedback features

This decision avoids introducing artificial relationships and preserves data integrity.

In [15]:
# STEP 3: Merge decision + analytical base

import numpy as np
import pandas as pd

# Ensure transaction_date is datetime
df_txn["transaction_date"] = pd.to_datetime(df_txn["transaction_date"], errors="coerce")

# Create _txn_month if not yet created
if "_txn_month" not in df_txn.columns:
    df_txn["_txn_month"] = df_txn["transaction_date"].dt.to_period("M").astype(str)

print("Merge decision:")
print("- Transactions are customer-level")
print("- No product_id exists in transaction data")
print("- Product offering data will be used only for recommendation mapping later")

# Draft base dataset = transactions only (for now)
df_base = df_txn.copy()
print("Base dataset shape:", df_base.shape)

df_base.head(3)

Merge decision:
- Transactions are customer-level
- No product_id exists in transaction data
- Product offering data will be used only for recommendation mapping later
Base dataset shape: (4900, 6)


,transaction_id,customer_id,transaction_date,transaction_amount,transaction_type,_txn_month
0,1,393,2023-01-01 00:00:00,3472.0,purchase,2023-01
2,3,916,2023-01-01 02:00:00,10.0,purchase,2023-01
3,4,109,2023-01-01 03:00:00,72.0,investment,2023-01


## Step 4: Customer-Level Feature Engineering

Clustering requires one row per customer. In this step, transaction-level records are aggregated into customer-level features.
These features aim to capture behavioural patterns such as monetary value, frequency, recency, and transaction type mix.

Customer feedback is also aggregated to capture satisfaction and advocacy signals that complement transactional behaviour.

## Step 4: Customer-Level Feature Engineering

Clustering requires one row per customer. In this step, transaction-level records are aggregated into customer-level features.
These features aim to capture behavioural patterns such as monetary value, frequency, recency, and transaction type mix.

Customer feedback is also aggregated to capture satisfaction and advocacy signals that complement transactional behaviour.

## Step 4: Customer-Level Feature Engineering

Clustering requires one row per customer. In this step, transaction-level records are aggregated into customer-level features.
These features aim to capture behavioural patterns such as monetary value, frequency, recency, and transaction type mix.

Customer feedback is also aggregated to capture satisfaction and advocacy signals that complement transactional behaviour.

In [17]:
# STEP 4.1: Customer-level transaction aggregates

reference_date = df_txn["transaction_date"].max()

cust_txn_features = df_txn.groupby("customer_id").agg(
    total_transaction_count=("transaction_id", "count"),
    total_transaction_amount=("transaction_amount", "sum"),
    avg_transaction_amount=("transaction_amount", "mean"),
    min_transaction_amount=("transaction_amount", "min"),
    max_transaction_amount=("transaction_amount", "max"),
    active_months=("_txn_month", "nunique"),
    last_transaction_date=("transaction_date", "max")
).reset_index()

cust_txn_features["transaction_frequency_per_month"] = (
    cust_txn_features["total_transaction_count"] /
    cust_txn_features["active_months"].replace(0, np.nan)
)

cust_txn_features["recency_days"] = (reference_date - cust_txn_features["last_transaction_date"]).dt.days

cust_txn_features.head()

,customer_id,total_transaction_count,total_transaction_amount,avg_transaction_amount,min_transaction_amount,max_transaction_amount,active_months,last_transaction_date,transaction_frequency_per_month,recency_days
0,1,6,16836.0,2806.0,156.0,4993.0,4,2023-07-02 03:00:00,1.500000,26
1,2,2,4907.0,2453.5,2057.0,2850.0,2,2023-05-21 23:00:00,1.000000,67
2,3,1,1538.0,1538.0,1538.0,1538.0,1,2023-02-27 23:00:00,1.000000,150
3,4,2,8295.0,4147.5,3559.0,4736.0,2,2023-06-30 01:00:00,1.000000,28
4,5,5,14798.0,2959.6,1508.0,4878.0,3,2023-07-27 00:00:00,1.666667,1


In [18]:
# STEP 4.2: Transaction type distribution (proportions)

txn_type_counts = (
    df_txn.groupby(["customer_id", "transaction_type"])
          .size()
          .unstack(fill_value=0)
)

txn_type_prop = txn_type_counts.div(txn_type_counts.sum(axis=1), axis=0)
txn_type_prop.columns = [f"txn_type_prop_{c}" for c in txn_type_prop.columns]
txn_type_prop = txn_type_prop.reset_index()

txn_type_prop.head()

,customer_id,txn_type_prop_bill payment,txn_type_prop_investment,txn_type_prop_loan payment,txn_type_prop_purchase
0,1,0.5,0.166667,0.166667,0.166667
1,2,0.5,0.000000,0.500000,0.000000
2,3,1.0,0.000000,0.000000,0.000000
3,4,0.0,0.000000,0.500000,0.500000
4,5,0.4,0.200000,0.000000,0.400000


In [19]:
# STEP 4.3: Customer-level feedback aggregates

cust_feedback_features = df_fb.groupby("customer_id").agg(
    avg_satisfaction_score=("satisfaction_score", "mean"),
    avg_likelihood_to_recommend=("likelihood_to_recommend", "mean"),
    feedback_count=("satisfaction_score", "count")
).reset_index()

cust_feedback_features.head()

,customer_id,avg_satisfaction_score,avg_likelihood_to_recommend,feedback_count
0,1,8.500000,9.000000,2
1,2,4.333333,5.000000,3
2,3,8.333333,3.666667,3
3,4,5.500000,3.333333,6
4,5,6.444444,5.666667,9


In [20]:
# STEP 4.4: Merge all customer-level features

df_features = (
    cust_txn_features
    .merge(txn_type_prop, on="customer_id", how="left")
    .merge(cust_feedback_features, on="customer_id", how="left")
)

print("Final engineered customer feature table shape:", df_features.shape)
df_features.head(10)

Final engineered customer feature table shape: (993, 17)


,customer_id,total_transaction_count,total_transaction_amount,avg_transaction_amount,min_transaction_amount,max_transaction_amount,active_months,last_transaction_date,transaction_frequency_per_month,recency_days,txn_type_prop_bill payment,txn_type_prop_investment,txn_type_prop_loan payment,txn_type_prop_purchase,avg_satisfaction_score,avg_likelihood_to_recommend,feedback_count
0,1,6,16836.0,2806.000000,156.0,4993.0,4,2023-07-02 03:00:00,1.500000,26,0.500000,0.166667,0.166667,0.166667,8.500000,9.000000,2
1,2,2,4907.0,2453.500000,2057.0,2850.0,2,2023-05-21 23:00:00,1.000000,67,0.500000,0.000000,0.500000,0.000000,4.333333,5.000000,3
2,3,1,1538.0,1538.000000,1538.0,1538.0,1,2023-02-27 23:00:00,1.000000,150,1.000000,0.000000,0.000000,0.000000,8.333333,3.666667,3
3,4,2,8295.0,4147.500000,3559.0,4736.0,2,2023-06-30 01:00:00,1.000000,28,0.000000,0.000000,0.500000,0.500000,5.500000,3.333333,6
4,5,5,14798.0,2959.600000,1508.0,4878.0,3,2023-07-27 00:00:00,1.666667,1,0.400000,0.200000,0.000000,0.400000,6.444444,5.666667,9
5,6,3,8694.0,2898.000000,2749.0,3021.0,2,2023-04-21 04:00:00,1.500000,98,0.000000,0.333333,0.666667,0.000000,2.666667,3.333333,3
6,7,2,3572.0,1786.000000,545.0,3027.0,2,2023-07-18 03:00:00,1.000000,10,0.000000,0.000000,1.000000,0.000000,5.250000,4.400000,4
7,8,6,12252.0,2042.000000,60.0,4681.0,3,2023-04-25 06:00:00,2.000000,94,0.166667,0.000000,0.666667,0.166667,7.500000,4.333333,2
8,9,8,17320.0,2165.000000,241.0,4789.0,5,2023-06-04 22:00:00,1.600000,53,0.125000,0.375000,0.250000,0.250000,7.000000,4.000000,6
9,10,6,15682.0,2613.666667,529.0,4867.0,4,2023-07-22 17:00:00,1.500000,5,0.500000,0.166667,0.000000,0.333333,5.000000,3.500000,4


In [21]:
# STEP 4.5: Quick quality checks

print("Missingness (%):")
display((df_features.isna().mean() * 100).sort_values(ascending=False).to_frame("missing_pct").head(20))

print("\nNumeric summary:")
display(df_features.describe().T)


Missingness (%):


,missing_pct
customer_id,0.0
recency_days,0.0
avg_likelihood_to_recommend,0.0
avg_satisfaction_score,0.0
txn_type_prop_purchase,0.0
txn_type_prop_loan payment,0.0
txn_type_prop_investment,0.0
txn_type_prop_bill payment,0.0
transaction_frequency_per_month,0.0
total_transaction_count,0.0



Numeric summary:


,count,mean,min,25%,50%,75%,max,std
customer_id,993.0,500.871098,1.0,251.0,502.0,750.0,1000.0,288.630791
total_transaction_count,993.0,4.934542,1.0,3.0,5.0,6.0,13.0,2.158709
total_transaction_amount,993.0,15307.199396,76.0,7632.0,11818.0,16146.0,489516.0,33674.460932
avg_transaction_amount,993.0,3071.564449,76.0,2014.0,2457.875,2948.0,144144.0,6983.98957
min_transaction_amount,993.0,1015.244713,10.0,324.0,745.0,1369.0,4993.0,952.071749
max_transaction_amount,993.0,7025.695871,76.0,3592.0,4347.0,4734.0,480300.0,32704.960832
active_months,993.0,3.554884,1.0,3.0,4.0,4.0,7.0,1.274462
last_transaction_date,993,2023-06-18 11:59:09.244712960,2023-01-11 08:00:00,2023-06-01 18:00:00,2023-06-29 08:00:00,2023-07-16 11:00:00,2023-07-28 07:00:00,NaN
transaction_frequency_per_month,993.0,1.382897,1.0,1.0,1.333333,1.6,3.0,0.367627
recency_days,993.0,39.325277,0.0,11.0,28.0,56.0,197.0,37.046695


## Step 5: Export Cleaned and Engineered Datasets

Two outputs are exported for Week 2 submission and team alignment:
1) Cleaned transaction-level dataset
2) Customer-level engineered feature dataset (one row per customer), ready for Week 3 validation and clustering preparation

In [23]:
# STEP 5: Export

OUTPUT_TXN = "finmark_cleaned_transactions.csv"
OUTPUT_FEAT = "finmark_engineered_customer_features.csv"

df_txn.to_csv(OUTPUT_TXN, index=False)
df_features.to_csv(OUTPUT_FEAT, index=False)

print("Exported:")
print(OUTPUT_TXN)
print(OUTPUT_FEAT)

Exported:
finmark_cleaned_transactions.csv
finmark_engineered_customer_features.csv
